In [1]:
import yaml
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from datetime import timedelta, date

from macrosynergy.management.dq import DataQueryInterface

import os
print(f"Current working directory: {os.getcwd()}")
path_to_config = '..//data//'

Current working directory: C:\Users\Ralph\OneDrive\Documents\Technology\notebooks\applications


In [2]:
cids_dmca = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'SEK', 'USD']  # DM currency areas
cids_dmec = ['DEM', 'ESP', 'FRF', 'ITL', 'NLG']  # DM euro area countries
cids_latm = ['ARS', 'BRL', 'COP', 'CLP', 'MXN', 'PEN']  # Latam countries
cids_emea = ['HUF', 'ILS', 'PLN', 'RON', 'RUB', 'TRY', 'ZAR']  # EMEA countries
cids_emas = ['CNY', 'HKD', 'IDR', 'INR', 'KRW', 'MYR', 'PHP', 'SGD', 'THB', 'TWD']  # EM Asia countries
cids_dm = cids_dmca + cids_dmec
cids_em = cids_latm + cids_emea + cids_emas
cids = sorted(cids_dm + cids_em)

cids_eufx = ['CHF', 'HUF', 'NOK', 'PLN', 'RON', 'SEK']  # EUR benchmark
cids_g2fx = ['GBP', 'RUB', 'TRY']  # dual benchmark
cids_usfx = ['AUD', 'BRL', 'CAD', 'CLP', 'CNY', 'COP', 'EUR', 'IDR', 'ILS', 'INR', 'JPY', 'KRW', 'MYR',
             'MXN', 'NZD', 'PEN', 'PHP', 'SGD', 'THB', 'TWD', 'ZAR']  # USD benchmark
cids_fx = cids_usfx + cids_eufx + cids_g2fx

In [3]:
def dq_tickers(tickers, metric='value', start_date = '2000-01-01', path = '..//data//'):
    
    """
    Returns standardized dataframe of specified base tickers and metric
    
    
    :param <str> tickers: JPMaQS ticker of form <cid>_<xcat>.
    :param <str> metrics: must choose one or more from 'value', 'eop_lag', 'mop_lag', or 'grade'. Default is ['value'].
    :param <str> start_date: first date in ISO 8601 string format. 
    :param <str> path: relative path from notebook to credential files.
    
    :return <pd.Dataframe> standardized dataframe with columns 'cid', 'xcats', 'real_date' and chosen metrics.
    """


    unique_tix = list(set(tickers))
    unique_tix.sort()
    dq_tix = ["DB(JPMAQS," + tick + f",{metric})" for tick in unique_tix]
    
    with open(f"{path}config.yml", 'r') as f:
        cf = yaml.load(f, Loader=yaml.FullLoader)

    with DataQueryInterface(
                username=cf["dq"]["username"],
                password=cf["dq"]["password"],
                crt=f"{path}api_macrosynergy_com.crt",
                key=f"{path}api_macrosynergy_com.key") as dq:
        
        print("Connected: ", dq.check_connection())
        df = dq.get_ts_expression(expression=dq_tix, original_metrics=metric, start_date=start_date)
        
        if isinstance(df, pd.DataFrame):
            df = df.sort_values(['cid', 'xcat', 'real_date']).reset_index(drop=True)

    return df

In [4]:
def dq_download(tickers=None, xcats=None, cids=None, metrics=['value'], start_date = '2000-01-01', path = '..//data//'):
    
    """
    Returns standardized dataframe of specified base tickers and metrics
        
    :param <List[str]> tickers: JPMaQS ticker of form <cid>_<xcat>. Can be combined with selection of categories.
    :param <List[str]> xcats: JPMaQS category codes. Downloaded for all standard cross sections identifiers available 
    (if cids are not specified) or those selected (if cids are specified).
    Standard cross sections here include major developed and emerging currency markets. See JPMaQS documentation.
    :param <List[str]> cids: JPMaQS cross-section identifiers, typically based  on currency code. See JPMaQS documentation.
    :param <str> metrics: must choose one or more from 'value', 'eop_lag', 'mop_lag', or 'grade'. Default is ['value'].
    # Todo: allow multiple metrics to be downloaded
    :param <str> start_date: first date in ISO 8601 string format. 
    :param <str> path: relative path from notebook to credential files.
    
    :return <pd.Dataframe> standardized dataframe with columns 'cid', 'xcats', 'real_date' and chosen metrics.
    """
    
    # A. Collect all standard cross sections.
    
    if (cids is None) & (xcats is not None):
    
        cids_dmca = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'SEK', 'USD']  # DM currency areas
        cids_dmec = ['DEM', 'ESP', 'FRF', 'ITL', 'NLG']  # DM euro area countries
        cids_latm = ['BRL', 'COP', 'CLP', 'MXN', 'PEN']  # Latam countries
        cids_emea = ['HUF', 'ILS', 'PLN', 'RON', 'RUB', 'TRY', 'ZAR']  # EMEA countries
        cids_emas = ['CNY', 'IDR', 'INR', 'KRW', 'MYR', 'PHP', 'SGD', 'THB', 'TWD']  # EM Asia countries
        cids_dm = cids_dmca + cids_dmec
        cids_em = cids_latm + cids_emea + cids_emas
        cids = sorted(cids_dm + cids_em)  # standard default
        
    
    # B. Collect all tickers and metrics to be downloaded
    
    dl = {'tickers': tickers, 'xcats': xcats, 'cids': cids, 'metrics': metrics}
    for key, value in dl.items():
        if isinstance(value, str):
            exec(f"{key} = [{key}]")  # make sure all choices are iterables
        
    if tickers is None:
        tickers = []
    
    if xcats is not None:
        add_tix = [cid + '_' + xcat for xcat in xcats for cid in cids]
        tickers = tickers + add_tix
    
    df = pd.DataFrame(columns = ['cid', 'xcat', 'real_date', 'value'])
    for metric in metrics:
        df_add = dq_tickers(tickers, metric=metric, start_date = start_date, path = path)
        df.append(df_add)
        
    return df

In [5]:
cats = ['CPIXFE_SJA_P6M6ML6AR'] 
tix = [cid + '_' + cat for cid in cids for cat in cats]

start = timer()
df = dq_download(tickers=tix, metrics=['value'], xcats = 'DU05YXR_NSA', start_date='2000-01-01')  # import via API
end = timer()

print("Download time from DQ: "+str(timedelta(seconds=end - start)))
print("Last updated:", date.today())

Connected:  True


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



exc_type: <class 'TypeError'>
exc_value: can only concatenate list (not "str") to list
exc_traceback: <traceback object at 0x00000255C2FA0440>


TypeError: can only concatenate list (not "str") to list

In [6]:
df

NameError: name 'df' is not defined

In [13]:
print(df.shape)
print(df.head())
print(df['cid'].unique())
print(df['xcat'].unique())

(56980, 4)
   cid                       xcat  real_date     value
0  AUD  INTRGDP_NSA_P1M1ML12_3MMA 2000-01-03  4.221025
1  AUD  INTRGDP_NSA_P1M1ML12_3MMA 2000-01-04  4.221025
2  AUD  INTRGDP_NSA_P1M1ML12_3MMA 2000-01-05  4.221025
3  AUD  INTRGDP_NSA_P1M1ML12_3MMA 2000-01-06  4.221025
4  AUD  INTRGDP_NSA_P1M1ML12_3MMA 2000-01-07  4.221025
['AUD' 'CAD' 'CHF' 'EUR' 'GBP' 'JPY' 'NOK' 'NZD' 'SEK' 'USD']
['INTRGDP_NSA_P1M1ML12_3MMA']
